In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# 0) Вывести ключевые S3A-настройки
hc = spark.sparkContext._jsc.hadoopConfiguration()
def h(key): 
    return hc.get(key)

print("endpoint         =", h("fs.s3a.endpoint"))
print("path.style       =", h("fs.s3a.path.style.access"))
print("ssl              =", h("fs.s3a.connection.ssl.enabled"))
print("credentials prov =", h("fs.s3a.aws.credentials.provider"))
print("access.key set?  =", bool(h("fs.s3a.access.key")))
print("secret.key set?  =", bool(h("fs.s3a.secret.key")))

# 1) Листинг в бакете (замени при необходимости)
BUCKET = "s3a://warehouse/"
jvm = spark._jvm
Path = jvm.org.apache.hadoop.fs.Path
FS = jvm.org.apache.hadoop.fs.FileSystem
uri = jvm.java.net.URI(BUCKET)
fs = FS.get(uri, hc)

print("\nСписок в", BUCKET)
for st in fs.listStatus(Path(BUCKET)):
    print("-", st.getPath().toString())

endpoint         = http://minio:9000
path.style       = true
ssl              = false
credentials prov = 
    org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider,
    org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider,
    com.amazonaws.auth.EnvironmentVariableCredentialsProvider,
    org.apache.hadoop.fs.s3a.auth.IAMInstanceCredentialsProvider
  
access.key set?  = True
secret.key set?  = True


25/08/24 07:04:09 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties



Список в s3a://warehouse/
- s3a://warehouse/iceberg


In [4]:
spark

In [6]:
TABLE_PATH = "s3a://warehouse/iceberg/bronze/crpt_2025_raw"
meta = f"{TABLE_PATH}/metadata"

# 1) Есть ли metadata/ — признак таблицы Iceberg
print("Проверяем наличие", meta)
exists = fs.exists(Path(meta))
print("metadata/ exists:", bool(exists))

# 2) Быстрое чтение 5 строк
df = spark.read.format("iceberg").load(TABLE_PATH)
df.limit(5).show(truncate=False)

Проверяем наличие s3a://warehouse/iceberg/bronze/crpt_2025_raw/metadata
metadata/ exists: True


+------+----------+-----+------------------------+-------------+------+-----------------+---------------------+----------------------------------+-----------+---------------------+-------------------------+-------------------+---------------------+-----------------------+
|Неделя|Год недели|Месяц|Тип эмиссии             |gtin         |Серия |ИНН производителя|ИНН вывода из оборота|Идентификатор МД вывода из оборота|Тип выбытия|Тип вывода из оборота|Выведено кодов маркировки|Выведено, руб с НДС|Выведено, руб без НДС|Источник финансирования|
+------+----------+-----+------------------------+-------------+------+-----------------+---------------------+----------------------------------+-----------+---------------------+-------------------------+-------------------+---------------------+-----------------------+
|10    |2025      |3    |Иностранное производство|3838989758257|LA3080|SI82646716       |5405074320           |00000000496491                    |Продажа    |NULL                 |1

In [11]:
# структура: s3a://warehouse/iceberg/bronze/crpt_2025_raw
spark.sql("SHOW NAMESPACES IN ice").show(truncate=False)
spark.sql("SHOW TABLES IN ice.bronze").show(truncate=False)
spark.table("ice.bronze.crpt_2025_raw").limit(5).show(truncate=False)

+---------+
|namespace|
+---------+
|bronze   |
+---------+

+---------+-------------+-----------+
|namespace|tableName    |isTemporary|
+---------+-------------+-----------+
|bronze   |crpt_2025_raw|false      |
|bronze   |crpt_load_log|false      |
+---------+-------------+-----------+

+------+----------+-----+------------------------+-------------+------+-----------------+---------------------+----------------------------------+-----------+---------------------+-------------------------+-------------------+---------------------+-----------------------+
|Неделя|Год недели|Месяц|Тип эмиссии             |gtin         |Серия |ИНН производителя|ИНН вывода из оборота|Идентификатор МД вывода из оборота|Тип выбытия|Тип вывода из оборота|Выведено кодов маркировки|Выведено, руб с НДС|Выведено, руб без НДС|Источник финансирования|
+------+----------+-----+------------------------+-------------+------+-----------------+---------------------+----------------------------------+-----------+------

In [8]:
# Снапшоты и файлы (работает и по каталожному имени, и по полному имени)
tbl = "ice.bronze.crpt_2025_raw"
spark.sql(f"SELECT snapshot_id, committed_at, operation FROM {tbl}.snapshots ORDER BY committed_at DESC").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|8012012217108634249|2025-08-23 23:42:42.169|append   |
|3629424066847997642|2025-08-23 23:39:31.663|append   |
|484499927572824358 |2025-08-23 23:36:44.261|append   |
|2286615430862358207|2025-08-23 23:33:46.991|append   |
|2473604785124089023|2025-08-23 23:32:10.965|append   |
|211446294242441037 |2025-08-23 23:29:11.883|append   |
|3466752973977639435|2025-08-23 23:26:29.18 |append   |
|285044371247057607 |2025-08-23 23:24:05.869|append   |
|6890475041206907799|2025-08-23 23:21:43.235|append   |
|2031417170101462200|2025-08-23 23:19:00.752|append   |
|1133908811693098057|2025-08-23 23:16:32.812|append   |
|4140667713898054888|2025-08-23 23:14:08.148|append   |
|4213941939151711478|2025-08-23 23:11:27.243|append   |
|5009047799226429782|2025-08-23 23:09:00.867|append   |
|452598316202781071 |2025-08-23 23:06:29.287|app

In [9]:
spark.sql(f"SELECT count(*) AS data_files, sum(file_size_in_bytes) AS size_bytes FROM {tbl}.files").show()

+----------+----------+
|data_files|size_bytes|
+----------+----------+
|      2344|9543587350|
+----------+----------+



In [13]:
spark.stop()
spark = SparkSession.builder.getOrCreate()

hc  = spark.sparkContext._jsc.hadoopConfiguration()
jvm = spark._jvm
Path = jvm.org.apache.hadoop.fs.Path
FS   = jvm.org.apache.hadoop.fs.FileSystem

def fs_for(uri_str: str):
    return FS.get(jvm.java.net.URI(uri_str), hc)

# выбери бакет, куда тестируешь запись
TEST_DIR = "s3a://raw/_connect_test"
# TEST_DIR = "s3a://warehouse/_connect_test"  # альтернативно

# 1) запись
import time, random
TEST_DIR = f"{TEST_DIR}_{int(time.time())}_{random.randint(0,9999)}"
spark.range(3).write.mode("overwrite").parquet(TEST_DIR)
print("Записали в:", TEST_DIR)

# 2) чтение
df = spark.read.parquet(TEST_DIR)
print("Прочитали обратно, count =", df.count())

# 3) удаление тем же FS, из того же бакета
fs = fs_for(TEST_DIR)              # <— ВАЖНО: FS для этого URI
ok = fs.delete(Path(TEST_DIR), True)
print("Удалили?", bool(ok))


25/08/24 07:23:23 WARN DataSource: All paths were ignored:                      
  s3a://raw/_connect_test_1756020199_8196


Записали в: s3a://raw/_connect_test_1756020199_8196
Прочитали обратно, count = 3
Удалили? True


In [14]:
# Проверим, что виден каталог и неймспейсы
print("defaultCatalog =", spark.conf.get("spark.sql.defaultCatalog"))
spark.sql("SHOW NAMESPACES IN ice").show(truncate=False)

# Создадим неймспейс и тестовую таблицу — проверим запись коммитов Iceberg
spark.sql("CREATE NAMESPACE IF NOT EXISTS ice.tmp")
spark.sql("DROP TABLE IF EXISTS ice.tmp.__ctest")

spark.sql("""
  CREATE TABLE ice.tmp.__ctest (id BIGINT)
  USING iceberg
  PARTITIONED BY (bucket(8, id))
""")
spark.sql("INSERT INTO ice.tmp.__ctest SELECT id FROM range(0, 10)")

print("count =", spark.table("ice.tmp.__ctest").count())
spark.sql("SELECT snapshot_id, committed_at, operation FROM ice.tmp.__ctest.snapshots ORDER BY committed_at DESC").show(truncate=False)

# Чисто — можно удалить
spark.sql("DROP TABLE ice.tmp.__ctest")

defaultCatalog = ice
+---------+
|namespace|
+---------+
|bronze   |
+---------+



count = 10
+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|8674144843526871029|2025-08-24 07:27:21.417|append   |
+-------------------+-----------------------+---------+



DataFrame[]

In [15]:
spark.sql("SHOW TABLES IN ice.bronze").show(truncate=False)
spark.table("ice.bronze.crpt_2025_raw").limit(5).show(truncate=False)
# т.к. defaultCatalog=ice, можно писать короче:
spark.table("bronze.crpt_2025_raw").limit(5).show(truncate=False)

# Расположение
spark.sql("DESCRIBE TABLE EXTENDED bronze.crpt_2025_raw").where("col_name='Location'").show(truncate=False)

+---------+-------------+-----------+
|namespace|tableName    |isTemporary|
+---------+-------------+-----------+
|bronze   |crpt_2025_raw|false      |
|bronze   |crpt_load_log|false      |
+---------+-------------+-----------+

+------+----------+-----+------------------------+-------------+------+-----------------+---------------------+----------------------------------+-----------------------------------------------------+---------------------+-------------------------+-------------------+---------------------+-----------------------+
|Неделя|Год недели|Месяц|Тип эмиссии             |gtin         |Серия |ИНН производителя|ИНН вывода из оборота|Идентификатор МД вывода из оборота|Тип выбытия                                          |Тип вывода из оборота|Выведено кодов маркировки|Выведено, руб с НДС|Выведено, руб без НДС|Источник финансирования|
+------+----------+-----+------------------------+-------------+------+-----------------+---------------------+------------------------------

In [16]:
# Снапшоты
spark.sql("SELECT snapshot_id, committed_at, operation FROM bronze.crpt_2025_raw.snapshots ORDER BY committed_at DESC").show(truncate=False)

# Файлы (понять масштаб/партиции)
spark.sql("""
  SELECT content, record_count, file_size_in_bytes, file_path
  FROM bronze.crpt_2025_raw.files
  ORDER BY file_size_in_bytes DESC
  LIMIT 20
""").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|8012012217108634249|2025-08-23 23:42:42.169|append   |
|3629424066847997642|2025-08-23 23:39:31.663|append   |
|484499927572824358 |2025-08-23 23:36:44.261|append   |
|2286615430862358207|2025-08-23 23:33:46.991|append   |
|2473604785124089023|2025-08-23 23:32:10.965|append   |
|211446294242441037 |2025-08-23 23:29:11.883|append   |
|3466752973977639435|2025-08-23 23:26:29.18 |append   |
|285044371247057607 |2025-08-23 23:24:05.869|append   |
|6890475041206907799|2025-08-23 23:21:43.235|append   |
|2031417170101462200|2025-08-23 23:19:00.752|append   |
|1133908811693098057|2025-08-23 23:16:32.812|append   |
|4140667713898054888|2025-08-23 23:14:08.148|append   |
|4213941939151711478|2025-08-23 23:11:27.243|append   |
|5009047799226429782|2025-08-23 23:09:00.867|append   |
|452598316202781071 |2025-08-23 23:06:29.287|app

In [17]:
spark.table("ice.bronze.crpt_load_log").show(truncate=False)

+---------------+--------------------------+
|zip_name       |loaded_at                 |
+---------------+--------------------------+
|2025_19.csv.zip|2025-08-23 22:51:16.794433|
|2025_1.csv.zip |2025-08-23 22:25:27.292049|
|2025_26.csv.zip|2025-08-23 23:11:27.321692|
|2025_6.csv.zip |2025-08-23 23:33:47.076412|
|2025_30.csv.zip|2025-08-23 23:24:05.941055|
|2025_25.csv.zip|2025-08-23 23:09:00.946394|
|2025_2.csv.zip |2025-08-23 22:53:44.859   |
|2025_7.csv.zip |2025-08-23 23:36:44.350645|
|2025_29.csv.zip|2025-08-23 23:19:00.82884 |
|2025_22.csv.zip|2025-08-23 23:01:37.702308|
|2025_20.csv.zip|2025-08-23 22:56:27.43579 |
|2025_13.csv.zip|2025-08-23 22:34:52.515156|
|2025_18.csv.zip|2025-08-23 22:48:48.784363|
|2025_24.csv.zip|2025-08-23 23:06:29.366374|
|2025_10.csv.zip|2025-08-23 22:28:20.401748|
|2025_4.csv.zip |2025-08-23 23:29:11.971912|
|2025_16.csv.zip|2025-08-23 22:43:16.186677|
|2025_12.csv.zip|2025-08-23 22:34:17.600667|
|2025_23.csv.zip|2025-08-23 23:04:05.300402|
|2025_5.cs

In [18]:
spark.sql("""
SELECT CAST(summary['total-records'] AS BIGINT) AS total_rows
FROM bronze.crpt_2025_raw.snapshots
ORDER BY committed_at DESC
LIMIT 1
""").show()

+----------+
|total_rows|
+----------+
|1574418370|
+----------+



In [20]:
spark.sql("""
SELECT CAST(summary['total-records'] AS BIGINT) AS total_rows
FROM bronze.crpt_load_log.snapshots
ORDER BY committed_at DESC
LIMIT 1
""").show()

+----------+
|total_rows|
+----------+
|        31|
+----------+

